In [1]:
features = ['normal', 'dry', 'oily', 'combination', 'acne', 'sensitive', 'fine lines', 'wrinkles', 'redness',
            'dull', 'pore', 'pigmentation', 'blackheads', 'whiteheads', 'blemishes', 'dark circles', 'eye bags', 'dark spots','dark', 'light', 'mid-dark', 'mid-light']

In [2]:
def generate_vector(skin_type, concerns,skin_tone):
    vector = [0] * len(features)

    # Set skin type
    if skin_type in features[:5]:
        vector[features.index(skin_type)] = 1

    # Set concerns
    for concern in concerns:
        if concern in features[4:18]:
            vector[features.index(concern)] = 1

    if skin_tone in features[18:]:
        vector[features.index(skin_tone)] = 1

    return vector




# Example usage
skin_type = 'oily'
concerns = ['Level_2', 'redness']

acne_l = ''

for i in concerns:
    if i in ['Level_1','Level_2','Level_3']:
        acne_l = i
        concerns.append('acne')
        break

total_concerns = concerns + ['general care','sun protection']
skin_tone = 'dark'
x = generate_vector(skin_type, concerns,skin_tone)
print(x)


[0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]


In [13]:

from IPython.utils.tempdir import TemporaryWorkingDirectory
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

df2 = pd.read_csv('final_with_skin_tone.csv')
#makeup = pd.read_csv('makeup_final.csv')
entries = len(df2)
LABELS = list(df2.label.unique())
# features

features = ['normal', 'dry', 'oily', 'combination', 'acne', 'sensitive', 'fine lines', 'wrinkles', 'redness',
            'dull', 'pore', 'pigmentation', 'blackheads', 'whiteheads', 'blemishes', 'dark circles', 'eye bags', 'dark spots','dark', 'light', 'mid-dark', 'mid-light']



# utility functions

def search_concern(target, i):
    if target in df2.iloc[i]['concern']:
        return True
    return False


def name2index(name):
    return df2[df2["name"] == name].index.tolist()[0]


def index2prod(index):
    return df2.iloc[index]


def wrap(info_arr):
    result = {}
#     print(info_arr)
    result['brand'] = info_arr[0]
    result['name'] = info_arr[1]
    result['price'] = info_arr[2]
    result['url'] = info_arr[3]
    result['img'] = info_arr[4]
    result['skin type'] = info_arr[5]
    result['concern'] = str(info_arr[6]).split(',')
    result['skin tone'] = info_arr[7]
    return result

def wrap_makeup(info_arr):
    result = {}
#     print(info_arr)
    result['brand'] = info_arr[0]
    result['name'] = info_arr[1]
    result['price'] = info_arr[2]
    result['url'] = info_arr[3]
    result['img'] = info_arr[4]
    result['skin type'] = info_arr[5]

    return result

one_hot_encodings = np.zeros([entries, len(features)])


#skin types first
for i in range(entries):
    for j in range(5):
        target = features[j]
        sk_type = df2.iloc[i]['skin type']
        if sk_type == 'all':
            one_hot_encodings[i][0:5] = 1
        elif target == sk_type:
            one_hot_encodings[i][j] = 1

#other features
for i in range(entries):
    for j in range(4, len(features)):
        feature = features[j]
        if feature in df2.iloc[i]['concern']:
            one_hot_encodings[i][j] = 1

for i in range(entries):
    for j in range(18, len(features)):
        target = features[j]
        sk_type = df2.iloc[i]['skin tone']
        if sk_type == 'all':
            one_hot_encodings[i][18:22] = 1
        elif target == sk_type:
            one_hot_encodings[i][j] = 1

# recommend top 5 similar items from a category


def recs_cs(vector=None, name=None, label=None, count=5):
    if name:
        idx = name2index(name)
        fv = one_hot_encodings[idx]
    elif vector:
        fv = vector

    cs_values = cosine_similarity(np.array([fv, ]), one_hot_encodings)
    df2['cs'] = cs_values[0]

    if label:
        dff = df2[df2['label'] == label]
    else:
        dff = df2

    if name:
        dff = dff[dff['name'] != name]

    recommendations = dff.sort_values('cs', ascending=False).head(count)

    # Convert to DataFrame
    recommendations_df = recommendations[['brand', 'name', 'price', 'url', 'img', 'skin type', 'concern', 'skin tone']]

    return recommendations_df


    # overall recommendation


def recs_essentials(vector=None, name=None):
    response = {}
    for label in LABELS:
        if name:
            r = recs_cs(None, name, label)
        elif vector:
            r = recs_cs(vector, None, label)
        response[label] = r

    # Concatenate all recommendations into a single DataFrame
    all_recommendations = pd.concat(response.values(), keys=response.keys(), names=['Label', 'Index'])

    return all_recommendations





In [4]:
x = recs_essentials(x, None)

In [5]:
for i,j in x.items():
    for k in j:
        if acne_l != '':
            k['concern'].append(acne_l)
        k['concern'] = [x for x in set(k['concern']) if x != '']
        if any(con in k['concern'] for con in total_concerns):
            print(i,k)

face-moisturisers {'brand': 'ustraa', 'name': 'men oily skin moisturising cream 100g', 'price': '₹ 224', 'url': 'https://www.myntra.com/face-moisturisers/ustraa/ustraa-men-oily-skin-moisturising-cream-100g/13220398/buy', 'img': 'https://assets.myntassets.com/h_720,q_90,w_540/v1/assets/images/13220398/2021/3/15/2395b329-d3a9-4b57-a8bf-2a59f51019781615809377964-Ustraa-Men-Oily-Skin-Moisturising-Cream-100g-620161580937701-1.jpg', 'skin type': 'oily', 'concern': ['excess oil', 'Level_2', 'smoothening', 'softening'], 'skin tone': 'dark'}
face-moisturisers {'brand': 'skinkraft', 'name': 'customized moisturizer for oily skin - 45 ml', 'price': '₹ 499', 'url': 'https://www.myntra.com/face-moisturisers/skinkraft/skinkraft-customized-moisturizer-for-oily-skin---45-ml/15345052/buy', 'img': 'https://assets.myntassets.com/h_720,q_90,w_540/v1/assets/images/15345052/2021/9/2/00a22bbe-f828-47c0-aed6-6a68b431fcfa1630588600147SKINKRAFT1.jpg', 'skin type': 'oily', 'concern': ['hydration', 'excess oil', '

In [6]:
print(acne_l)

Level_2


In [7]:
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model('model.keras')

In [8]:
from tensorflow.keras.preprocessing import image
import numpy as np

def load_and_preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0
    return img_array

def predict_skin_type_acne_tone(model, img_array):
    predictions = model.predict(img_array)
    return predictions

# Example usage
img_path = '//Users//arushreemishra//Downloads//pic.jpg.jpg'
img_array = load_and_preprocess_image(img_path)
predictions = predict_skin_type_acne_tone(model, img_array)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


In [9]:
def convert_predictions_to_labels(predictions, skin_type_labels, acne_type_labels, skin_tone_labels):
    skin_type_probs = predictions[:, :len(skin_type_labels)]
    acne_type_probs = predictions[:, len(skin_type_labels):len(skin_type_labels) + len(acne_type_labels)]
    skin_tone_probs = predictions[:, len(skin_type_labels) + len(acne_type_labels):]

    skin_type_label = skin_type_labels[np.argmax(skin_type_probs)]
    acne_type_label = acne_type_labels[np.argmax(acne_type_probs)]
    skin_tone_label = skin_tone_labels[np.argmax(skin_tone_probs)]

    return skin_type_label, acne_type_label, skin_tone_label

# Define your labels
skin_type_labels = ['oily', 'dry', 'normal']
acne_type_labels = ['Level_0', 'Level_1', 'Level_2']
skin_tone_labels = ['light', 'mid-light', 'mid-dark','dark']

# Convert the predictions to human-readable labels
skin_type_label, acne_type_label, skin_tone_label = convert_predictions_to_labels(predictions, skin_type_labels, acne_type_labels, skin_tone_labels)


In [10]:
concerns = ['acne', 'redness']
vector = generate_vector(skin_type_label, concerns, skin_tone_label)

In [14]:
# Get recommendations
recommendations = recs_cs(vector=vector, count=5)
print("Top 5 Recommendations:", recommendations)

# Get overall recommendations across all labels
all_recommendations = recs_essentials(vector=vector)
print("All Recommendations:", all_recommendations)

Top 5 Recommendations:              brand                                               name  price  \
442  khadi natural  ayurvedic neem & tea tree sustainable face was...  ₹ 220   
667       fabindia                       neem tulsi face pack - 100 g  ₹ 360   
506        garnier  men acno fight anti-pimple for acne prone skin...  ₹ 213   
590          vedix  avasya clarifying clay mask for acne prone ski...  ₹ 399   
296  the body shop  the body shop vitamin e range tea tree skin cl...  ₹ 633   

                                                   url  \
442  https://www.myntra.com/face-wash-and-cleanser/...   
667  https://www.myntra.com/mask-and-peel/fabindia/...   
506  https://www.myntra.com/face-wash-and-cleanser/...   
590  https://www.myntra.com/mask-and-peel/vedix/ved...   
296  https://www.myntra.com/face-wash-and-cleanser/...   

                                                   img skin type  \
442  https://assets.myntassets.com/h_720,q_90,w_540...      oily   
667  https:

In [15]:
#example usage
def project():
    img_path = input("Enter the image file path: ")
    concerns = list(input("Write your concerns (comma-separated): ").split(','))

    img_array = load_and_preprocess_image(img_path)
    predictions = predict_skin_type_acne_tone(model, img_array)
    skin_type_label, acne_type_label, skin_tone_label = convert_predictions_to_labels(predictions, skin_type_labels, acne_type_labels, skin_tone_labels)
    vector = generate_vector(skin_type_label, concerns, skin_tone_label)

    option = input("Write the number of top recommendations or type 'all' for all recommendations: ").strip().lower()
    if option == 'all':
        # Get all recommendations
        all_recommendations = recs_essentials(vector=vector)
        print("All Recommendations:", all_recommendations)
    else:
        try:
            # Get top recommendations (convert option to an integer if valid)
            count = int(option)
            recommendations = recs_cs(vector=vector, count=count)
            print(f"Top {count} Recommendations:", recommendations)
        except ValueError:
            print("Invalid input for number of recommendations. Please enter a valid number or 'all'.")


In [16]:
project()

Enter the image file path:  //Users//arushreemishra//Downloads//acne-face-8806301_1280.jpg
Write your concerns (comma-separated):  acne,blackheads,dark spots


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step


Write the number of top recommendations or type 'all' for all recommendations:  2


Top 2 Recommendations:         brand                                               name   price  \
35   biotique  bio winter green spot correcting anti-acne sus...   ₹ 117   
229       phy  men set of green tea face wash - scrub & moist...  ₹ 1200   

                                                   url  \
35   https://www.myntra.com/face-moisturisers/bioti...   
229  https://www.myntra.com/face-moisturisers/phy/p...   

                                                   img skin type  \
35   https://assets.myntassets.com/h_720,q_90,w_540...      oily   
229  https://assets.myntassets.com/h_720,q_90,w_540...      oily   

                                      concern skin tone  
35   acne,blemishes,dark spots,acne,blemishes      dark  
229       acne,blemishes,dark spots,hydration      dark  
